In [ ]:
#Setting up all environment prerequisites
!pip install llama-index
## Document loaders for SimpleDirectoryReader
!pip install docx2txt
!pip install pypdf
## Install openai
!pip install openai
import pandas as pd
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from IPython.display import display, HTML
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor
from llama_index.core.postprocessor.llm_rerank import LLMRerank
from collections import defaultdict
import os
import openai
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
from google.colab import userdata
api_key = userdata.get("OpenAI-Personal-Key")
client = openai.OpenAI(api_key = api_key)
openai.api_key = api_key

#Loading all the documents from the source folder
reader = SimpleDirectoryReader(input_dir="/content/drive/MyDrive/GenAI_Course_Master/Course_2_HelpMateAICodes/Insurance Policy Documents")
documents = reader.load_data()
print(f"Loaded {len(documents)} docs")

## Section for all Functions : BOC
def query_response(user_input,query_engine):
    response = query_engine.query(user_input)
    source_nodes = response.source_nodes

    if not source_nodes:
        return response.response + "\nI am unable to answer that accurately based on the data that has been presented to me."

    # Organize page labels by file_name
    file_pages_map = defaultdict(list)

    for node in source_nodes:
        metadata = node.node.metadata
        file_name = metadata.get('file_name', 'unknown file')
        page_label = metadata.get('page_label', 'unknown page')
        file_pages_map[file_name].append(page_label)

    # Check if all file names are the same
    if len(file_pages_map) == 1:
        reference_parts = []
        # Single file: list all pages together
        file_name = next(iter(file_pages_map))
        pages = ", ".join(file_pages_map[file_name])
        reference_parts.append(f"<b>File Name :</b>{file_name}  <b>(Page Nos :</b> {pages}<b>)</b>")
        reference_info = "<br>Check further at:<br>" + "<br>".join(reference_parts)
    else:
        # Multiple files: include pages for each file
        reference_parts = []
        for file_name, pages in file_pages_map.items():
            page_list = ", ".join(pages)
            reference_parts.append(f"<b>File Name :</b>{file_name}  <b>(Page Nos :</b> {page_list}<b>)</b>")
        reference_info = "<br>Check further at:<br>" + "<br>".join(reference_parts)

    final_response = response.response + reference_info +"<br><br>"
    return final_response

def initialize_conversation():
  # Initialize a query engine for the index with a specified similarity top-k value
  #using vector index retriever query engine
  retriever = VectorIndexRetriever(index=index,similarity_top_k=4)
  postprocessor=SimilarityPostprocessor(similarity_cutoff=0.82)
  query_engine=RetrieverQueryEngine(retriever=retriever,node_postprocessors=[postprocessor])
  #
  display(HTML(f"""
      <div style="font-size:22px; font-weight:bold; color:#1a73e8; margin-bottom:10px;">
        Feel free to ask Questions regarding different Insurance Policy Benefits and their Terms and Conditions.
      </div>
    """))
  #print('Feel free to ask Questions regarding different Insurance Policy Benefits and their Terms and Conditions.')
  while True:
    print()
    #user_input = input()
    user_input = input('Enter your question directly in the following box (or) type "exit" whenever you want to exit the conversation : \n')
    display(HTML(f"""
      <div style="font-size:22px; font-weight:bold; color:#1a73e8; margin-bottom:10px;">
        You asked: {user_input}
      </div>
    """))

    #display(HTML(f'<p style="font-size:20px">{user_input}</p>'))
    if user_input.lower() == 'exit':
      print('Exiting the program... bye')
      break
    else:
      response = query_response(user_input,query_engine)
      #display(HTML(f'<p style="font-size:20px">Chat Response : {response}</p>'))
      styled_html = f"""
        <div style="font-size:20px; font-family:Arial, sans-serif; line-height:1.6; color:#333;">
          <strong>Semantic Spotter Response:</strong><br>
          {response}
        </div>
      """
      display(HTML(styled_html))
      #display(HTML(f'<p style="font-size:20px">Chat Response : {response1}</p>'))

# Safely display any embedded HTML content in response, and wrap it with styled div

#Suggestion 1 - Using Customized Nodes and LLMs
##Initialize the OpenAI model
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=256)
##Initialize the embedding model
Settings.embed_model = OpenAIEmbedding()
## Initialize the node_parser with the custom node settings
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
## Initialize the num_output and the context window
Settings.num_output = 512
Settings.context_window = 3900

template_str= """
with this given {context_str} you have to answer the below
{query_str}, Always use the context only to answer, if the context is not helpful you just write no context found.
"""
text_qa_template = PromptTemplate(template_str)

PERSIST_DIR = "/content/drive/MyDrive/GenAI_Course_Master/Course_3_SemanticSpotter/persistentstore"
if not os.path.exists(PERSIST_DIR):
  #load the documents and create the index
# Create a VectorStoreIndex from a list of documents using the service context using persistent storage
  index = VectorStoreIndex.from_documents(documents)
  #save the index to disk
  index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
  #load the existing index
  storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
  index = load_index_from_storage(storage_context)

#Call the Conversation Method here
initialize_conversation()
